### Adafruit PCA9685 16-Channel Servo Tests

Checking out the servo comtrol board. You need to install the needed libraries using these commands in a terminal

```
sudo pip3 install adafruit-circuitpython-pca9685
sudo pip3 install adafruit-circuitpython-servokit
```



In [1]:
import time
import board
import busio
import adafruit_pca9685
from adafruit_servokit import ServoKit
import itertools

### Arm Controller Class

Simple class to control the arm.

In [2]:
class Joint:
    # init method or constructor
    def __init__(self, name, servo, home_position=90, max_value=180, min_value=0, speed=30):
         
        self.name = name
        self.servo = servo
        self.max_value = max_value
        self.min_value = min_value
        self.speed = speed
        self.home_position = home_position
        
        # Homing
        self.servo.angle = self.home_position

    # Output on print
    def __str__(self):
        return f"{self.name} at position {self.servo.angle:.1f}"
        
    # Move to angle at speed
    def move_to(self, angle):
    
        start  = self.servo.angle    
        points = abs(round(angle-start))
        if (points>0):
            delta  = (angle-start)/points
            for p in range(points):
                start=start+delta
                self.servo.angle = start
                time.sleep(1.0/self.speed)

        self.servo.angle = angle
        
    # Go home
    def home(self):
        self.move_to(self.home_position)
        

In [3]:
class Arm():  
    # init method or constructor
    def __init__(self):    
        i2c = busio.I2C(board.SCL, board.SDA)
        pca = adafruit_pca9685.PCA9685(i2c)
        self.kit = ServoKit(channels=16)
        
        self.joints = [ 
                        Joint("Rotation",       self.kit.servo[0], min_value = 10, max_value = 170, home_position = 90),
                        Joint("Lower arm",      self.kit.servo[1], min_value = 45, max_value = 140, home_position = 90),
                        Joint("Upper arm",      self.kit.servo[2], min_value = 20, max_value = 120, home_position = 90),
                        Joint("Wrist tilt",     self.kit.servo[3], min_value = 50, max_value = 175, home_position = 90),
                        Joint("Gripper",        self.kit.servo[4], min_value =  5, max_value = 170, home_position = 90)
                      ]
        
        self.locations = []

    def __str__(self):
        info = "Arm status\n"
        for joint in self.joints:
            info = info + "  " +str(joint) + "\n"
        return info
    
    def home(self):
        for joint in self.joints:
            joint.home()
            
    def move(self, name, angle):
        for joint in self.joints:
            if(joint.name == name):
                joint.move_to(angle)
                
    def goto(self, name, angle):
        for joint in self.joints:
            if(joint.name == name):
                joint.servo.angle = angle

    def clear_locations(self):
        self.locations.clear()
        
    def save_location(self):
        self.locations.append([joint.servo.angle for joint in self.joints])
        
    def run_locations(self):
        for location in self.locations:
            for (angle, joint) in zip(location, self.joints):
                joint.move_to(angle)
        
    def stored_locations(self):
        return len(self.locations)

### Arm Controller UI

In [4]:
import ipywidgets.widgets as widgets

In [5]:
# Instanciate the arm object
arm=Arm()

Callback from UI.

In [ ]:
def handle_slider_change(change):
    if change.new != change.old:
        arm.move(change.owner.description, change.new)

In [ ]:
def home_button_pressed(button):
    arm.home()
    update_sliders()

In [ ]:
def save_button_pressed(button):
    arm.save_location()
    message.value = f"Stored points: {arm.stored_locations()}"

In [ ]:
def reset_button_pressed(button):
    arm.clear_locations()
    message.value = f"Stored points: {arm.stored_locations()}"

In [ ]:
def play_button_pressed(button):
    arm.run_locations()
    message.value = f"Done"

In [ ]:
def update_sliders():
    for i, joint in enumerate(arm.joints):
        sliders[i].value = joint.servo.angle

In [ ]:
sliders=[]
for joint in arm.joints:
    sliders.append(widgets.FloatSlider(value=joint.servo.angle, 
                                       min=joint.min_value, max=joint.max_value, step=1.0, description=joint.name,
                                       disabled=False, continuous_update=False, orientation='horizontal',
                                       readout=True, readout_format='.1f',layout=widgets.Layout(width='90%', height='50px')))
    sliders[-1].observe(handle_slider_change, names='value')

In [ ]:
home_button = widgets.Button(description='Home', button_style='danger', layout=widgets.Layout(width='80%', height='40px'))
home_button.on_click(home_button_pressed)
save_button = widgets.Button(description='Save', button_style='success', layout=widgets.Layout(width='80%', height='40px'))
save_button.on_click(save_button_pressed)
reset_button = widgets.Button(description='Reset', button_style='warning', layout=widgets.Layout(width='80%', height='40px'))
reset_button.on_click(reset_button_pressed)
play_button = widgets.Button(description='Play', button_style='danger', layout=widgets.Layout(width='80%', height='40px'))
play_button.on_click(play_button_pressed)
message = widgets.Label(value="-", layout=widgets.Layout(align_self='center'))
message.value = f"Stored points: {arm.stored_locations()}"

In [ ]:
panel_sliders = widgets.VBox(sliders, layout=widgets.Layout(width='80%'))
panel_buttons = widgets.VBox([home_button, save_button, reset_button, play_button, message],layout=widgets.Layout(width='15%'))
display(widgets.HBox([panel_sliders,panel_buttons]))

### Game Controller

Do not run opther widgets or performance gets super bad :)

In [ ]:
controller = widgets.Controller(index=0)
display(controller)

In [ ]:
def onChangeGripper(change):
    if abs(change.new - change.old) > 0.001:
        new = 90 + 45*change.new
        arm.move("Gripper", new)

def onChangeWrist(change):
    if abs(change.new - change.old) > 0.001:
        new = 90 + 25*change.new
        arm.move("Wrist tilt", new)
        
controller.axes[0].observe(onChangeWrist, names='value')
controller.axes[1].observe(onChangeGripper, names='value')

### Using GameContoller natively

This needs the code from https://github.com/piborg/Gamepad and the controller dongle is plugged into the middle USB port of the Raspberry Pi.

In [6]:
# Load the gamepad and time libraries
import Gamepad
import time

In [ ]:
# Gamepad settings
running = True

# Wait for a connection
if not Gamepad.available():
    print('Please connect your gamepad...')
    while not Gamepad.available():
        time.sleep(1.0)
        
gamepad = Gamepad.Android()
print('Gamepad connected')

# Handle joystick updates one at a time
while running and gamepad.isConnected():
    # Wait for the next event
    eventType, control, value = gamepad.getNextEvent()

    # Determine the type
    if eventType == 'BUTTON':
        if control == 'X':
            # Exit button (event on press)
            if value:
                print('EXIT')
                running = False
                
    elif eventType == 'AXIS':
        # Joystick changed
        if control == 'LEFT-Y':
            new = 90 + 45*value
            arm.goto("Gripper", new)
        elif control == 'LEFT-X':
            new = 90 + 45*value
            arm.goto("Wrist tilt", new)
        elif control == 'RIGHT-Y':
            new = 90 + 45*value
            arm.goto("Upper arm", new)
        elif control == 'RIGHT-X':
            new = 90 + 45*value
            arm.goto("Lower arm", new)
        elif control =='DPAD-X':
            new = 
    
    #print(f"{eventType}, {control}, {value}")

Gamepad connected


In [ ]:
arm.home()